## Select Stimulus (Video or Image)
### Select the stimulus video or image file.

In [ ]:
import importlib
import os
import sys
from pathlib import Path

import ipyfilechooser
import pymovements
from IPython.display import display

import videoreplay.video_player
from videoreplay.video_player import VideoPlayer

# Step 1: Create the file chooser widget
base_dir = os.getcwd()
while not base_dir.endswith("pymovements-videoreplay") and os.path.dirname(base_dir) != base_dir:
    # Go up one level until we find the project root
    base_dir = os.path.dirname(base_dir)

default_path = os.path.join(base_dir, "data")
os.makedirs(default_path, exist_ok=True)

chooser = ipyfilechooser.FileChooser(default_path)
chooser.filter_pattern = ['*.mp4', '*.jpg', '*.jpeg', '*.png']
display(chooser)

# Step 2: Function to move the selected file


def save_uploaded_stimulus():
    selected_file_path = chooser.selected  # Get selected file path
    if selected_file_path is None:
        print("ERROR: No stimulus selected. Please select an image or video.")
        return

    # Verify the file exists
    stimulus_name = os.path.basename(selected_file_path)
    if os.path.exists(selected_file_path):
        print(f"Stimulus saved successfully as '{stimulus_name}'!")
    else:
        print("ERROR: Stimulus file was not saved correctly!")

    return selected_file_path

In [ ]:
# Step 3: After Selecting the File, Run:
selected_stimulus_path = save_uploaded_stimulus()


current_path = Path().resolve()
for parent in current_path.parents:
    if parent.name == "pymovements-videoreplay":
        project_root = parent
        break
else:
    raise RuntimeError("Project root 'pymovements-videoreplay' not found!")

dataset_path = project_root / "data" / "SB-SAT" / "fixation"
selected_dataset_path = dataset_path.resolve()

### Or download a public dataset from pymovements

In [ ]:
def select_public_dataset() -> str | None:
    available_datasets = pymovements.datasets.__all__
    if not available_datasets:
        print("No public datasets available from pymovements.")
        return None

    dataset_prompt = "Select a public dataset by number:\n\n"
    for i, name in enumerate(available_datasets):
        dataset_prompt += f"  {i + 1}: {name}\n"
    dataset_prompt += "\nYour choice: "

    try:
        choice = int(input(dataset_prompt)) - 1
        if 0 <= choice < len(available_datasets):
            return available_datasets[choice]
        else:
            print("Invalid selection.")
    except ValueError:
        print("Invalid input. Please enter a number.")

    return None

In [ ]:
dataset = None

dataset_name = select_public_dataset()
if dataset_name is not None:
    dataset = pymovements.Dataset(dataset_name, f'data/{dataset_name}')
    dataset.download()
    dataset.load()
    print(f"Successfully loaded public dataset: {dataset_name}")
else:
    print("ERROR: No dataset loaded. Please select a dataset first.")

if dataset is not None:
    selected_dataset_path = dataset.paths.precomputed_events
    # selected_stimulus_path = dataset.paths.stimuli
    print(selected_dataset_path)

## Initialize and Run VideoPlayer
Run the VideoPlayer to visualize eye-tracking data overlay.

In [ ]:
# Ensure 'src' is in sys.path
src_path = os.path.abspath(os.path.join('src'))
if src_path not in sys.path:
    sys.path.append(src_path)

# Import and Reload the module
importlib.reload(videoreplay.video_player)

# Now import the updated class

# Initialize the VideoPlayer with the uploaded video
player = VideoPlayer(stimulus_path=selected_stimulus_path,
                     dataset_path=selected_dataset_path,
                     recording_sessions=["msd001", "msd002"])

## Play Video with Gaze Overlay

In [ ]:
player.play(speed=1.0)

## Navigate Through Fixations
Use 'n' for next fixation, 'p' for previous, 'q' to quit.

In [ ]:
player.fixation_navigation()

## Export Gaze Replay as MP4
Save the replay with gaze overlay.

In [ ]:
player.export_replay("gaze_replay", speed=1.0)